In [7]:
#path = '/home/jack/sampler/data/08_reporting/11 - Simulator 200 points with Interest + Coverage (normalized)/increased_data.csv'
path = '/home/jack/sampler/data/08_reporting/12 - Simulator 100 points with p(interest) * Std (lots of outliers)/increased_data.csv'

# Load the data into a pandas dataframe
data = pd.read_csv(path)

data.tail()

,r_ext_pAl,r_ext_pMeO,pAl_richness,Pg_f,Tg_Tmax,sim_time,Pg_f_hat,Tg_Tmax_hat,std,interest,coverage
295,0.070500,0.814139,0.208426,0.287017,0.650126,25.000575,0.293472,0.637852,0.684524,0.114383,0.0
296,0.071742,0.465603,0.183253,0.312714,0.663947,25.000818,0.295406,0.605133,0.619962,0.162088,0.0
297,0.133646,0.531250,0.482029,0.247170,0.624225,25.000320,0.262173,0.612996,0.465825,0.278873,0.0
298,0.174916,0.462056,0.518675,0.284974,0.644785,25.000053,0.285395,0.643081,0.365401,0.309209,0.0
299,0.234837,0.432848,0.625000,0.292023,0.668388,25.000216,0.288788,0.654127,0.332093,0.328055,0.0


In [8]:
p = catalog.load('parameters')

features=p["features"]
targets=p["targets"]
additional_values = p["additional_values"]
names=p["names"]

scaler=catalog.load('scaler')
treatment=catalog.load('treatment')

[02/19/24 13:29:20] INFO     Loading data from 'parameters' (MemoryDataSet)...                  ]8;id=524404;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=245583;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'scaler' (PickleDataSet)...                      ]8;id=769784;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=903197;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'treatment' (PickleDataSet)...                   ]8;id=26636;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=538816;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [9]:
interest_region = p['interest_region']
interest_region

{'Pg_f': [1000000.0, 2000000.0], 'Tg_Tmax': [2000, 4500]}

In [10]:
from sampler.common.data_treatment import scale_interest_region

In [11]:
scaled_interest_region = scale_interest_region(features, interest_region, scaler)
scaled_interest_region


{
    'Pg_f': [0.24311482097304826, 0.34781867528634947],
    'Tg_Tmax': [0.4766833551336145, 0.7977511856164203]
}

In [13]:
# Add column 'interest' with True if the row is in the interest region
data['is_interest_point'] = (data['Pg_f'] >= scaled_interest_region['Pg_f'][0]) & \
                            (data['Pg_f'] <= scaled_interest_region['Pg_f'][1]) & \
                            (data['Tg_Tmax'] >= scaled_interest_region['Tg_Tmax'][0]) & \
                            (data['Tg_Tmax'] <= scaled_interest_region['Tg_Tmax'][1])

In [14]:
data

,r_ext_pAl,r_ext_pMeO,pAl_richness,Pg_f,Tg_Tmax,sim_time,Pg_f_hat,Tg_Tmax_hat,std,interest,coverage,is_interest_point
0,0.998455,0.752775,0.832825,0.439480,0.861510,NaN,NaN,NaN,NaN,NaN,NaN,False
1,0.884572,0.995134,0.489310,0.590641,0.910139,NaN,NaN,NaN,NaN,NaN,NaN,False
2,0.773458,0.891320,0.651214,0.499086,0.883874,NaN,NaN,NaN,NaN,NaN,NaN,False
3,0.513232,0.514477,0.939324,0.321652,0.782492,NaN,NaN,NaN,NaN,NaN,NaN,True
4,0.304354,0.935717,0.096649,0.685096,0.909678,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.070500,0.814139,0.208426,0.287017,0.650126,25.000575,0.293472,0.637852,0.684524,0.114383,0.0,True
296,0.071742,0.465603,0.183253,0.312714,0.663947,25.000818,0.295406,0.605133,0.619962,0.162088,0.0,True
297,0.133646,0.531250,0.482029,0.247170,0.624225,25.000320,0.262173,0.612996,0.465825,0.278873,0.0,True
298,0.174916,0.462056,0.518675,0.284974,0.644785,25.000053,0.285395,0.643081,0.365401,0.309209,0.0,True


In [17]:
data[(data['is_interest_point'] == True) & (data['sim_time'] > 25)]

,r_ext_pAl,r_ext_pMeO,pAl_richness,Pg_f,Tg_Tmax,sim_time,Pg_f_hat,Tg_Tmax_hat,std,interest,coverage,is_interest_point
109,0.401399,1.000000,0.990153,0.267700,0.703047,25.000292,0.274722,0.759037,0.530698,0.265654,0.0,True
113,0.125000,0.812500,0.342983,0.319843,0.660493,25.000055,0.296328,0.570261,0.585267,0.042667,0.0,True
126,0.102206,0.713869,0.289240,0.323731,0.662119,25.000344,0.293789,0.617429,0.771600,0.001817,0.0,True
127,0.104261,0.656250,0.279882,0.332806,0.665805,25.000170,0.293749,0.580156,0.771114,0.001857,0.0,True
128,0.146413,0.562500,0.437500,0.293592,0.648911,25.000072,0.277744,0.597922,0.793363,0.008184,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.070500,0.814139,0.208426,0.287017,0.650126,25.000575,0.293472,0.637852,0.684524,0.114383,0.0,True
296,0.071742,0.465603,0.183253,0.312714,0.663947,25.000818,0.295406,0.605133,0.619962,0.162088,0.0,True
297,0.133646,0.531250,0.482029,0.247170,0.624225,25.000320,0.262173,0.612996,0.465825,0.278873,0.0,True
298,0.174916,0.462056,0.518675,0.284974,0.644785,25.000053,0.285395,0.643081,0.365401,0.309209,0.0,True


In [18]:
data[(data['is_interest_point'] == True) & (data['sim_time'] < 25)]

,r_ext_pAl,r_ext_pMeO,pAl_richness,Pg_f,Tg_Tmax,sim_time,Pg_f_hat,Tg_Tmax_hat,std,interest,coverage,is_interest_point
100,0.281360,0.875000,0.750000,0.279198,0.675718,9.649067,0.278098,0.716604,0.869652,0.002753,0.0,True
102,0.237305,1.000000,0.687500,0.271329,0.651600,11.136867,0.282267,0.708161,0.831927,0.006889,0.0,True
103,0.437484,0.686521,1.000000,0.278129,0.721351,7.214238,0.278602,0.749832,0.854351,0.005708,0.0,True
104,0.437500,0.686507,1.000000,0.278106,0.721433,7.211467,0.278607,0.749838,0.854356,0.005703,0.0,True
107,0.339388,0.753602,0.895023,0.266900,0.682699,14.462489,0.269641,0.699022,0.922530,0.000334,0.0,True
116,0.216419,0.250000,0.500000,0.332207,0.686868,22.307778,0.298610,0.578032,0.323627,0.155420,0.0,True
122,0.201385,0.622621,0.468750,0.334004,0.692495,24.671186,0.296331,0.635764,0.411904,0.147548,0.0,True
123,0.207190,0.582388,0.500000,0.323594,0.686493,24.591614,0.295462,0.643843,0.389692,0.156871,0.0,True
124,0.211175,0.625000,0.500000,0.327322,0.691114,24.081994,0.295423,0.647482,0.393150,0.155553,0.0,True
125,0.218750,0.506093,0.579675,0.297514,0.668588,24.748486,0.289277,0.662802,0.355822,0.173973,0.0,True


In [24]:
# Add column interest_per_batch with the mean interest points from last 7 rows
batch = 14
data['interest_per_batch'] = data['is_interest_point'].rolling(batch).mean()
data.head()

,r_ext_pAl,r_ext_pMeO,pAl_richness,Pg_f,Tg_Tmax,sim_time,Pg_f_hat,Tg_Tmax_hat,std,interest,coverage,is_interest_point,interest_per_batch
0,0.998455,0.752775,0.832825,0.439480,0.861510,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
1,0.884572,0.995134,0.489310,0.590641,0.910139,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
2,0.773458,0.891320,0.651214,0.499086,0.883874,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
3,0.513232,0.514477,0.939324,0.321652,0.782492,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN
4,0.304354,0.935717,0.096649,0.685096,0.909678,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN


In [25]:
# Use plotly to plot interest_per_batch
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data['interest_per_batch'], mode='lines', name='Interest per batch'))

fig.show()
